#Environment Configuration, Please Ignore.

In [1]:
import os
from google.colab import drive

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

In [2]:
!git clone https://github.com/Yichuan0712/tabula-data-yichuan.git

Cloning into 'tabula-data-yichuan'...
remote: Enumerating objects: 1356, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 1356 (delta 184), reused 201 (delta 158), pack-reused 1054 (from 1)
Receiving objects: 100% (1356/1356), 1.23 MiB | 5.16 MiB/s, done.
Resolving deltas: 100% (681/681), done.


In [3]:
# !cp /content/drive/MyDrive/OSU/tabula_data.env /content/.env
!cp /content/drive/MyDrive/OSU/yichuan_gemini.env /content/.env

In [4]:
!pip install python-dotenv -q
!pip install fake_useragent -q
!pip install shortuuid -q
!pip install ratelimit -q
!pip install langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:00


In [5]:
import os
os.chdir('/content/tabula-data-yichuan')

In [6]:
from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
if "GEMINI_15_API_KEY" in os.environ:
    genai.configure(api_key=os.environ.get("GEMINI_15_API_KEY", None))

#This tutorial begins here.


###First, prepare an HTML table (a single table) along with its corresponding description (caption + footnote).

In [7]:
html_content = """
<section id="S8">
   <h3 class="pmc_sec_title">Pharmacokinetics</h3>
   <p id="P25"><a class="usa-link" href="#T2">Table II</a> summarizes the noncompartmental PK evaluation for the Elective Cohort. Eight pharmacokinetic samples (0.3%) were excluded, seven due to suspected contamination from the infused drug and one due to collection during the lorazepam infusion. Overall, the mean area-under-the-curve (AUC<sub>0–∞</sub>) was 822.5 ng*hr/mL and the median AUC<sub>0–∞</sub> was 601.5 ng*hr/mL with an average dose of 0.04 mg/kg. The overall fit of the population PK model was good over the wide range of individuals in the population. There were no covariates meeting criteria for inclusion into the model. Thirty three subjects (23 Status Cohort, 10 Elective Cohort) had received, at baseline, at least one agent that can induce drug metabolizing enzymes. The calculated value for terminal (beta) half-life was 16 hours for a typical 24 kg child. The empiric Bayesian estimated parameters from the post-hoc analysis are summarized in <a class="usa-link" href="#T3">Table III</a>. The model demonstrated good model prediction of observed concentrations even when applied to the patients exhibiting the highest and lowest individual clearances, and in a patient who received a total of 9 doses of lorazepam during the PK sampling interval.</p>
   <section class="tw xbox font-sm" id="T2">
      <h4 class="obj_head">Table 2.</h4>
      <div class="caption p">
         <p id="P40">Non-compartmental pharmacokinetics parameters from Elective Cohort patients. C<sub>max</sub> is maximum concentration. AUC<sub>0−∞</sub> is area-under-the-curve to infinity. CL is clearance. Vdz is apparent volume of distribution. T<sub>1/2</sub> is half-life.</p>
      </div>
      <div class="tbl-box p" tabindex="0">
         <table class="content" frame="hsides" rules="groups">
            <thead>
               <tr>
                  <th align="left" colspan="1" rowspan="1"></th>
                  <th align="center" colspan="1" rowspan="1">C<sub>max</sub><br/>(ng/mL)</th>
                  <th align="center" colspan="1" rowspan="1" valign="top">AUC<sub>0−∞</sub> </th>
                  <th align="center" colspan="1" rowspan="1">CL<br/>(mL/min/kg)</th>
                  <th align="center" colspan="1" rowspan="1">CL<br/>(mL/min/m<sup>2</sup>)</th>
                  <th align="center" colspan="1" rowspan="1">Vdz<br/>(L/kg)</th>
                  <th align="center" colspan="1" rowspan="1">T<sub>1/2</sub><br/>(hr)</th>
               </tr>
            </thead>
            <tbody>
               <tr>
                  <td align="left" colspan="1" rowspan="1">N</td>
                  <td align="center" colspan="1" rowspan="1">15</td>
                  <td align="center" colspan="1" rowspan="1">15</td>
                  <td align="center" colspan="1" rowspan="1">15</td>
                  <td align="center" colspan="1" rowspan="1">15</td>
                  <td align="center" colspan="1" rowspan="1">15</td>
                  <td align="center" colspan="1" rowspan="1">15</td>
               </tr>
               <tr>
                  <td align="left" colspan="1" rowspan="1" valign="top">Range</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">29.3–209.6</td>
                  <td align="center" colspan="1" rowspan="1">253.3–3202.5</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">3.33–131.50</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">5.5–67.5</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">0.33–4.05</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">9.5–47.0</td>
               </tr>
               <tr>
                  <td align="left" colspan="1" rowspan="1">Mean ± s.d.</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">56.1 ± 44.9</td>
                  <td align="center" colspan="1" rowspan="1">822.5 ± 706.1</td>
                  <td align="center" colspan="1" rowspan="1">49.33 ± 30.83</td>
                  <td align="center" colspan="1" rowspan="1" valign="top">31.95 ± 13.99</td>
                  <td align="center" colspan="1" rowspan="1">1.92 ± 0.84</td>
                  <td align="center" colspan="1" rowspan="1">20.5 ± 10.2</td>
               </tr>
               <tr>
                  <td align="left" colspan="1" rowspan="1">Median</td>
                  <td align="center" colspan="1" rowspan="1">42.2</td>
                  <td align="center" colspan="1" rowspan="1">601.5</td>
                  <td align="center" colspan="1" rowspan="1">41.50</td>
                  <td align="center" colspan="1" rowspan="1">32.34</td>
                  <td align="center" colspan="1" rowspan="1">1.94</td>
                  <td align="center" colspan="1" rowspan="1">18.1</td>
               </tr>
            </tbody>
         </table>
      </div>
      <div class="p text-right font-secondary"><a class="usa-link" href="table/T2/" rel="noopener noreferrer" target="_blank">Open in a new tab</a></div>
   </section>
</section>
"""

In [8]:
caption_and_footnote = "Non-compartmental pharmacokinetics parameters from Elective Cohort patients. C<sub>max</sub> is maximum concentration. AUC<sub>0−∞</sub> is area-under-the-curve to infinity. CL is clearance. Vdz is apparent volume of distribution. T<sub>1/2</sub> is half-life."

###Second, use `single_html_table_to_markdown` to convert the HTML table into the required Markdown format. Note that no other tools should be used for Markdown conversion.

In [9]:
from TabFuncFlow.utils.table_utils import single_html_table_to_markdown

markdown_table = single_html_table_to_markdown(html_content)
print(markdown_table)

| Unnamed_0 | Cmax(ng/mL) | AUC0−∞ | CL(mL/min/kg) | CL(mL/min/m) | Vdz(L/kg) | T1/2(hr) |
| --- | --- | --- | --- | --- | --- | --- |
| N | 15 | 15 | 15 | 15 | 15 | 15 |
| Range | 29.3–209.6 | 253.3–3202.5 | 3.33–131.50 | 5.5–67.5 | 0.33–4.05 | 9.5–47.0 |
| Mean ± s.d. | 56.1 ± 44.9 | 822.5 ± 706.1 | 49.33 ± 30.83 | 31.95 ± 13.99 | 1.92 ± 0.84 | 20.5 ± 10.2 |
| Median | 42.2 | 601.5 | 41.50 | 32.34 | 1.94 | 18.1 |


###Finally, use `p_pk_summary` to extract information from the table.

In [10]:
from TabFuncFlow.pipelines.p_pk_summary import p_pk_summary

# llm = "chatgpt_4o"
llm = "gemini_15_pro"

# The difference between content_list and content_list_clean is that the latter retains only the reasoning process without including response on how to call functions.
dataframe_table_result, step_list, res_list, content_list, content_list_clean, usage_list, truncated_list = p_pk_summary(markdown_table, caption_and_footnote, llm)

Pre-launch Inspection
Usage: 0
Result:
Markdown Table:
col: | "Unnamed_0" | "Cmax(ng/mL)" | "AUC0−∞" | "CL(mL/min/kg)" | "CL(mL/min/m)" | "Vdz(L/kg)" | "T1/2(hr)" |
| --- | --- | --- | --- | --- | --- | --- |
row 0: | N | 15 | 15 | 15 | 15 | 15 | 15 |
row 1: | Range | 29.3–209.6 | 253.3–3202.5 | 3.33–131.50 | 5.5–67.5 | 0.33–4.05 | 9.5–47.0 |
row 2: | Mean ± s.d. | 56.1 ± 44.9 | 822.5 ± 706.1 | 49.33 ± 30.83 | 31.95 ± 13.99 | 1.92 ± 0.84 | 20.5 ± 10.2 |
row 3: | Median | 42.2 | 601.5 | 41.50 | 32.34 | 1.94 | 18.1 |
Description:
Non-compartmental pharmacokinetics parameters from Elective Cohort patients. C<sub>max</sub> is maximum concentration. AUC<sub>0−∞</sub> is area-under-the-curve to infinity. CL is clearance. Vdz is apparent volume of distribution. T<sub>1/2</sub> is half-life.
Reasoning:
Automatic execution.

Drug Information Extraction
Usage: 1027
Result:
col: | "Drug name" | "Analyte" | "Specimen" |
| --- | --- | --- |
row 0: | N/A | N/A | Plasma |
Reasoning:
Here's my thought

In [11]:
dataframe_table_result

,Drug name,Analyte,Specimen,Population,Pregnancy stage,Subject N,Parameter type,Unit,Value,Summary Statistics,Variation type,Variation value,Interval type,Lower limit,High limit,P value
0,N/A,N/A,Plasma,N/A,N/A,15,Apparent volume of distribution (Vdz),L/kg,1.92,Mean,Standard Deviation (SD),0.84,Range,0.33,4.05,N/A
1,N/A,N/A,Plasma,N/A,N/A,15,Apparent volume of distribution (Vdz),L/kg,1.94,Median,N/A,N/A,N/A,N/A,N/A,N/A
2,N/A,N/A,Plasma,N/A,N/A,15,Area-under-the-curve to infinity (AUC0−∞),ng/mL*hr,822.5,Mean,Standard Deviation (SD),706.1,Range,253.3,3202.5,N/A
3,N/A,N/A,Plasma,N/A,N/A,15,Area-under-the-curve to infinity (AUC0−∞),ng/mL*hr,601.5,Median,N/A,N/A,N/A,N/A,N/A,N/A
4,N/A,N/A,Plasma,N/A,N/A,15,Clearance (CL),mL/min/kg,49.33,Mean,Standard Deviation (SD),30.83,Range,3.33,131.50,N/A
5,N/A,N/A,Plasma,N/A,N/A,15,Clearance (CL),mL/min/kg,41.50,Median,N/A,N/A,N/A,N/A,N/A,N/A
6,N/A,N/A,Plasma,N/A,N/A,15,Clearance (CL),mL/min/m,31.95,Mean,Standard Deviation (SD),13.99,Range,5.5,67.5,N/A
7,N/A,N/A,Plasma,N/A,N/A,15,Clearance (CL),mL/min/m,32.34,Median,N/A,N/A,N/A,N/A,N/A,N/A
8,N/A,N/A,Plasma,N/A,N/A,15,Half-life (T1/2),hr,20.5,Mean,Standard Deviation (SD),10.2,Range,9.5,47.0,N/A
9,N/A,N/A,Plasma,N/A,N/A,15,Half-life (T1/2),hr,18.1,Median,N/A,N/A,N/A,N/A,N/A,N/A


In [12]:
step_list

['Pre-launch Inspection',
 'Drug Information Extraction',
 'Population Information Extraction',
 'Individual Data Deletion',
 'Parameter Type Alignment',
 'Column Header Categorization',
 'Rough Task Allocation',
 'Unit Extraction (Trial 1)',
 'Unit Extraction (Final)',
 'Drug Matching (Final)',
 'Population Matching (Final)',
 'Parameter Value Extraction (Trial 1)',
 'Parameter Value Extraction (Trial 2)',
 'Parameter Value Extraction (Trial 3)',
 'Parameter Value Extraction (Final)',
 'Assembly',
 'Post-Processing']

In [13]:
res_list

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [14]:
content_list

['Automatic execution.\n',
 'Here\'s my thought process:\n\n1. **Identify the drug:** The caption mentions "Non-compartmental pharmacokinetics parameters." This suggests a single drug is being studied. However, the drug name is not explicitly stated in the table or caption.  We need to look for more context.  Since there\'s no other information provided, we\'ll have to mark the drug name as "N/A" for now.\n\n2. **Identify the analyte:**  The caption defines C<sub>max</sub>, AUC<sub>0−∞</sub>, CL, Vdz, and T<sub>1/2</sub>. These are standard pharmacokinetic parameters typically referring to the *parent drug* unless otherwise specified (e.g., mentioning a metabolite). Since no metabolite is mentioned, it\'s reasonable to infer that the analyte is the same as the drug itself.  So, the analyte will also be "N/A" for now, mirroring the drug name.\n\n3. **Identify the specimen:** The table doesn\'t explicitly state the specimen type.  However, pharmacokinetic parameters like C<sub>max</sub>,

In [15]:
content_list_clean

['Automatic execution.\n',
 'Here\'s my thought process:\n\n1. **Identify the drug:** The caption mentions "Non-compartmental pharmacokinetics parameters." This suggests a single drug is being studied. However, the drug name is not explicitly stated in the table or caption.  We need to look for more context.  Since there\'s no other information provided, we\'ll have to mark the drug name as "N/A" for now.\n\n2. **Identify the analyte:**  The caption defines C<sub>max</sub>, AUC<sub>0−∞</sub>, CL, Vdz, and T<sub>1/2</sub>. These are standard pharmacokinetic parameters typically referring to the *parent drug* unless otherwise specified (e.g., mentioning a metabolite). Since no metabolite is mentioned, it\'s reasonable to infer that the analyte is the same as the drug itself.  So, the analyte will also be "N/A" for now, mirroring the drug name.\n\n3. **Identify the specimen:** The table doesn\'t explicitly state the specimen type.  However, pharmacokinetic parameters like C<sub>max</sub>,

In [16]:
usage_list

[0, 1027, 1047, 566, 630, 891, 0, 1197, 0, 0, 0, 1558, 1517, 1464, 0, 0, 0]

In [17]:
truncated_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]